In [1]:
import os

In [2]:
%pwd

'e:\\Neoron\\Programming_Practice\\Machine_Learning_Project\\winequality_prediction\\research'

In [3]:
os.chdir('../')
%pwd

'e:\\Neoron\\Programming_Practice\\Machine_Learning_Project\\winequality_prediction'

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [5]:
from src.WineQualityPrediction.constants import *
from src.WineQualityPrediction.utils.common import read_yaml, create_directories
from src.WineQualityPrediction.utils.my_logging import logger
from src.WineQualityPrediction.utils.my_exception import CustomException

In [6]:
class ConfigurationManager:
    """
    Manages the configuration for the project by reading YAML files
    and providing structured configurations.

    Attributes:
        config (ConfigBox): Parsed configuration file.
        params (ConfigBox): Parsed parameters file.
        schema (ConfigBox): Parsed schema file.
    """

    def __init__(self,
                 config_filepath: Path = CONFIG_FILE_PATH,
                 params_filepath: Path = PARAMS_FILE_PATH,
                 schema_filepath: Path = SCHEMA_FILE_PATH):
        """
        Initializes the ConfigurationManager by loading YAML files and creating directories.

        Args:
            config_filepath (Path): Path to the main configuration file.
            params_filepath (Path): Path to the parameters configuration file.
            schema_filepath (Path): Path to the schema configuration file.
        """
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        # Ensure the artifacts root directory exists
        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        """
        Provides the configuration for the data ingestion component.

        Returns:
            DataIngestionConfig: Configuration object for data ingestion.
        """
        config = self.config.data_ingestion
        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )
        return data_ingestion_config


In [14]:
import os
import sys
import urllib.request as request
from src.WineQualityPrediction.utils.my_logging import logger
import zipfile
import logging

In [8]:
log_path = 'log\log_file.log'

In [11]:
class DataIngestion:
    """
    Handles the ingestion of data, including downloading and extracting files.

    Attributes:
        config (DataIngestionConfig): Configuration object containing paths and URLs.
    """

    def __init__(self, config: DataIngestionConfig):
        """
        Initializes the DataIngestion instance with configuration.

        Args:
            config (DataIngestionConfig): Configuration for data ingestion.
        """
        self.config = config

    def download_file(self) -> None:
        """
        Downloads the file from the source URL to the local path if it doesn't already exist.
        """
        try:
            if not os.path.exists(self.config.local_data_file):
                filename, headers = request.urlretrieve(
                    url=self.config.source_URL,
                    filename=self.config.local_data_file
                )
                logger(log_path,logging.INFO,f"Downloaded file from {self.config.source_URL} to {filename}")
            else:
                logger(log_path,logging.INFO,f"File already exists at {self.config.local_data_file}")
        except Exception as e:
            logger(log_path,logging.ERROR,f"Error downloading file: {CustomException(e,sys)}")
            raise CustomException(e,sys)

    def extract_zip_file(self) -> None:
        """
        Extracts the ZIP file located at `self.config.local_data_file` 
        to the directory specified by `self.config.unzip_dir`.
        """
        try:
            unzip_path = self.config.unzip_dir
            os.makedirs(unzip_path, exist_ok=True)
            with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
                zip_ref.extractall(unzip_path)
            logger(log_path,logging.INFO,f"Extracted ZIP file to {unzip_path}")
        except zipfile.BadZipFile as e:
            logger(log_path,logging.ERROR,f"Error extracting the ZIP file: {CustomException(e,sys)}")
            raise CustomException(e,sys)
        except Exception as e:
            logger(log_path,logging.ERROR,f"Error extracting the ZIP file: {CustomException(e,sys)}")
            raise CustomException(e,sys)


In [13]:
try:
    config=ConfigurationManager()
    data_ingestion_config=config.get_data_ingestion_config()
    data_ingestion=DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-01-14 15:54:18] - INFO - Yaml file read successfully
[2025-01-14 15:54:18] - INFO - Yaml file read successfully
[2025-01-14 15:54:18] - INFO - Yaml file read successfully
[2025-01-14 15:54:18] - INFO - Directory created at artifacts
[2025-01-14 15:54:18] - INFO - Directory created at artifacts/data_ingestion
[2025-01-14 15:54:19] - INFO - Downloaded file from https://github.com/krishnaik06/datasets/raw/refs/heads/main/winequality-data.zip to artifacts/data_ingestion/data.zip
[2025-01-14 15:54:19] - INFO - Extracted ZIP file to artifacts/data_ingestion
